In [ ]:
# Montar Google Drive
from google.colab import drive
drive.mount('/content/drive')

# Cambiar el directorio de trabajo
import os
import sys

# Define the TFM directory path
TFM_PATH = '/content/drive/My Drive/TFM'

# Change the current working directory to the TFM directory
os.chdir(TFM_PATH)
print(f"Current working directory changed to: {os.getcwd()}")

# Add the TFM directory to the Python system path
if TFM_PATH not in sys.path:
    sys.path.append(TFM_PATH)
    print(f"'{TFM_PATH}' added to Python system path.")
else:
    print(f"'{TFM_PATH}' is already in Python system path.")

In [ ]:
from pathlib import Path
import pandas as pd
import re
import numpy as np

from scipy.stats import shapiro, normaltest, levene, fligner, bartlett, skew, kurtosis
from statsmodels.stats.multitest import multipletests

from sklearn.decomposition import PCA
from sklearn.covariance import MinCovDet

from scipy.cluster.hierarchy import linkage, fcluster, dendrogram
from scipy.spatial.distance import squareform

import matplotlib.pyplot as plt
import seaborn as sns

import extractData

In [ ]:
data_path = TFM_PATH + '/data/'
participant_data_path = data_path + 'participant_genetic_data/'
modelos = data_path + 'Modelos/'
std_files = data_path + 'std_files/'

download_files = "download_files.csv"

In [ ]:
def extraer_rsids_desde_xref(xref):
    """
    xRef suele venir como:
        'dbsnp.83:rs526642' o
        'dbsnp.100:rs2748067;dbsnp.131:rs76046194'

    Devolvemos un conjunto de rsID encontrados.
    """
    rsids = set()
    if not xref:
        return rsids

    # Separar por ';' y luego por ':' y quedarnos con las cadenas que empiezan por 'rs'
    for chunk in xref.split(";"):
        for piece in chunk.split(":"):
            piece = piece.strip()
            if piece.startswith("rs"):
                rsids.add(piece)
    return rsids


In [ ]:
def estandarizar_TSV_files(var_path, out_tsv_path,save_file = False):
    """
    Recorre el fichero de variantes de Complete Genomics y genera
    un TSV estandarizado con:

        rsID, chr, pos, effect_allele, other_allele,
        effect_weight, allele1, allele2, dosage
    """
    # rsID -> lista de alelos observados ['A','G'] (máx. ploidía)
    genotipos_data = [] # Usaremos una lista para acumular los datos

    with extractData.open_compressed_file(var_path, "rt") as f:
        # Saltar hasta la cabecera que empieza con '>'
        for line in f:
            if line.startswith(">"):
                break

        # A partir de aquí vienen las filas de variantes
        for line in f:
            line = line.rstrip("\n")
            if not line or line.startswith("#"):
                continue


            parts = line.split("\t")
            # El fichero debe tener 16 columnas según tu descripción
            #if len(parts) < 16:
            #    continue

            # Campos según el orden que comentabas
            # 0:locus, 1:ploidy, 2:allele, 3:chromosome, 4:begin, 5:end,
            # 6:varType, 7:reference, 8:alleleSeq, ..., 13:xRef, 14:alleleFreq, 15:alternativeCalls
            var_type = parts[6]
            allele_pos = parts[2]
            chrom = parts[3]
            pos = parts[5]   # cojo el end en lugoar de begin
            allele_ref = parts[7]
            allele_seq = parts[8]
            var_filter = parts[11]
            xref = parts[13]

            # Solo SNPs cuyo calidad sea buena (varFilter == PASS), ignoramos otras variantes
            if var_type.lower() != "snp":
                continue
            else:
                if (var_filter.lower() != "pass" and var_filter.lower()!="vqhigh"):
                    continue
            # Alelo desconocido
            if allele_seq in ("", ".", "?", "N"):
                continue

            rsids = [
                r for r in extraer_rsids_desde_xref(xref)
                #if r in pgs_dict
            ]

            if not rsids:
                # Ningún rsID de esta fila está en el modelo
                continue

            # Si hay más de un rsID, nos quedamos con el primero que está en el modelo
            rsid = rsids[0]

            # En función de si es el primer alelo o el segundo lo pone en allele1 o allele2
            if allele_pos == "1":
                a1_ref = allele_ref
                a1 = allele_seq
                a2 = a2_ref = '.'
            else:
                a2_ref = allele_ref
                a2 = allele_seq
                a1 = a1_ref = '.'


            genotipos_data.append({
                "rsID": rsid,
                "chr": chrom,
                "pos": pos,
                "allele1_ref": a1_ref,
                "allele2_ref": a2_ref,
                "allele1": a1,
                "allele2": a2,
                "dosage":1
            })

    # Crear el DataFrame de pandas con los datos acumulados
    df_genotipos = pd.DataFrame(genotipos_data)

    # Escribir el DataFrame estandarizado a un fichero TSV
    if (save_file):
        out_path = Path(out_tsv_path)
        df_genotipos.to_csv(out_tsv_path, sep="\t", index=False)

    return df_genotipos, out_tsv_path


In [ ]:
def estandarizar_TSV3_files(var_path, out_tsv_path,save_file = False):
    """
    Recorre el fichero de variantes de Complete Genomics y genera
    un TSV estandarizado con:

        rsID, chr, pos, effect_allele, other_allele,
        effect_weight, allele1, allele2, dosage
    """
    # rsID -> lista de alelos observados ['A','G'] (máx. ploidía)
    genotipos_data = [] # Usaremos una lista para acumular los datos

    with extractData.open_compressed_file(var_path, "rt") as f:
        # Saltar hasta la cabecera que empieza con '>'
        for line in f:
            if line.startswith(">"):
                break

        # A partir de aquí vienen las filas de variantes
        for line in f:
            line = line.rstrip("\n")
            if not line or line.startswith("#"):
                continue


            parts = line.split("\t")
            # El fichero debe tener 16 columnas según tu descripción
            #if len(parts) < 16:
            #    continue

            # Campos según el orden que comentabas
            # 0:locus, 1:ploidy, 2:chromosome, 3:begin, 4:end, 5:zigosity
            # 6:varType, 7:reference, 8:allele1Seq, 9:allele2Seq, ..., 14:allele1VarQuality,
            # 15:allele2VarQuality, 15:alternativeCalls,..., 18:allele1XRef, 19:allele2XRef, ...
            var_type = parts[6]
            chrom = parts[2]
            pos = parts[4]   # cojo el end en lugoar de begin
            allele_ref = parts[7]
            allele1_seq = parts[8]
            allele2_seq = parts[9]
            var1_filter = parts[14]
            var2_filter = parts[15]
            xref1 = parts[18]
            xref2 = parts[19]

            # Solo SNPs cuyo calidad sea buena (varFilter == PASS), ignoramos otras variantes
            if var_type.lower() != "snp":
                continue
            else:
                # Allele 1
                if var1_filter.lower() == "vqhigh":
                    if allele1_seq not in ("", ".", "?", "N"):
                        if (allele1_seq != allele_ref):
                            rsids = [r for r in extraer_rsids_desde_xref(xref1)]
                            if rsids:
                                rsid = rsids[0]
                                a1_ref = allele_ref
                                a1 = allele1_seq
                                a2 = a2_ref = '.'
                                genotipos_data.append({
                                    "rsID": rsid,
                                    "chr": chrom,
                                    "pos": pos,
                                    "allele1_ref": a1_ref,
                                    "allele2_ref": a2_ref,
                                    "allele1": a1,
                                    "allele2": a2,
                                    "dosage":1
                                })

                if var2_filter.lower() == "vqhigh":
                    if allele2_seq not in ("", ".", "?", "N"):
                        if (allele2_seq != allele_ref):
                            rsids = [r for r in extraer_rsids_desde_xref(xref2)]
                            if rsids:
                                rsid = rsids[0]
                                a2_ref = allele_ref
                                a2 = allele2_seq
                                a1 = a1_ref = '.'
                                genotipos_data.append({
                                    "rsID": rsid,
                                    "chr": chrom,
                                    "pos": pos,
                                    "allele1_ref": a1_ref,
                                    "allele2_ref": a2_ref,
                                    "allele1": a1,
                                    "allele2": a2,
                                    "dosage":1
                                })

    # Crear el DataFrame de pandas con los datos acumulados
    df_genotipos = pd.DataFrame(genotipos_data)

    # Escribir el DataFrame estandarizado a un fichero TSV
    if (save_file):
        out_path = Path(out_tsv_path)
        df_genotipos.to_csv(out_tsv_path, sep="\t", index=False)

    return df_genotipos, out_tsv_path


In [ ]:
def get_ac_value(info_string):
    """
    Busca 'AC' en una cadena de información y devuelve su valor. Si no se encuentra 'AC', devuelve 1.
    Ejemplo de cadena: 'AB=0.294118;ABP=15.5282;AC=2;ADP=41'
    """
    for item in info_string.split(';'):
        if item.startswith('AC='):
            try:
                # Extrae el valor después de 'AC='
                return int(item.split('=')[1])
            except ValueError:
                # En caso de que el valor de AC no sea un entero válido
                return 1 # O manejar el error de otra manera
    return 1



In [ ]:
def get_dosage_value(info_string):
    """
    Busca 'HET'; 'HOM' en una cadena de información y devuelve su valor. Si no se encuentra ninguna cadena, devuelve 0.
    Ejemplo de cadena: 'AB=0.294118;ABP=15.5282;AC=2;ADP=41'
    """
    dosage = 0
    het = 0
    hom = 0
    for item in info_string.split(';'):
        if item.startswith('HET='):
            try:
                het =  int(item.split('=')[1])
            except ValueError:
                het = 0
        if item.startswith('HOM='):
            try:
                hom =  int(item.split('=')[1])
            except ValueError:
                hom = 0
    if het == 1 and hom == 0:
        dosage = 1
    elif het == 0 and hom == 1:
        dosage = 2
    else:
        dosage = 0

    return dosage


In [ ]:
def parse_genotype(gt: str, ref: str, alt: str, info:str):
    """
    Convierte un campo GT (ej. '0/1', '1|1', './.') a (allele1, allele2)
    usando REF y ALT.

    Devuelve (allele1, allele2) donde cada uno es una base o '.' si missing.
    """

    if gt in (".", "./.", ".|."):
        return ".", ".",".","."

    # ALT puede tener múltiples alelos: 'A,G'
    alt_alleles = alt.split(",") if alt not in (".", "") else []

    # Separar por / o | (no nos importa la fase para PRS)
    tokens = re.split(r"[\/|]", gt)
    alleles = []
    alleles_ref =[]

    t = tokens[0]
    if t in ("", "."):
        alleles.append(".")
        alleles.append(".")
        alleles_ref.append(".")
        alleles_ref.append(".")

    try:
      idx = int(t)
    except ValueError:
        alleles.append(".")
        alleles.append(".")
        alleles_ref.append(".")
        alleles_ref.append(".")

    if idx == 0:
        alleles_ref.append(ref)
        alleles_ref.append(".")
        alleles.append(alt_alleles[idx])
        alleles.append(".")
    elif idx == 1:
        alleles_ref.append(".")
        alleles_ref.append(ref)
        alleles.append(".")
        alleles.append(alt_alleles[idx-1])
    else:
      alleles.append(".")
      alleles.append(".")
      alleles_ref.append(".")
      alleles_ref.append(".")

    count = get_ac_value(info)

    return alleles_ref[0], alleles_ref[1],alleles[0], alleles[1],count


In [ ]:
def estandarizar_VCF_files(var_path:str, out_vcf_path:str,
    add_chr_prefix: bool = False,save_file:bool=False,):
    """
    Convierte un VCF a un TSV con formato:
        rsID    chr    pos    allele1    allele2

    - vcf_path: ruta del .vcf o .vcf.gz
    - out_path: ruta del .tsv de salida
    - sample_name: nombre de la muestra (columna VCF).
        - Si es None, se usa la primera muestra que aparezca tras FORMAT.
    - add_chr_prefix: si True, convierte '1' -> 'chr1', etc.
    """

    # rsID -> lista de alelos observados ['A','G'] (máx. ploidía)
    genotipos_data = [] # Usaremos una lista para acumular los datos

    with extractData.open_compressed_file(var_path, "rt") as f:
        header_seen = False
        sample_idx = None

        # Recorre todas las líneas del fichero
        for line in f:
            line = line.rstrip("\n")

            # Ignora los metadatos del VCF
            if line.startswith("##"):
                continue

            # Cabecera de las columnas
            if line.startswith("#CHROM"):
                header_seen = True
                cols = line.lstrip("#").split("\t")

            # Las primeras 9 columnas son fijas en VCF
            # CHROM POS ID REF ALT QUAL FILTER INFO FORMAT [SAMPLES...]
            # Analizando datos cabecera
            if len(cols) < 10:
                print(
                    "La cabecera del VCF no tiene columnas de muestras.",
                    file=sys.stderr,
                )
                break;

            format_idx = 8  # índice de FORMAT
            sample_idx = 9

            # Si aún no hemos visto la cabecera, algo va mal
            if not header_seen:
                continue

            # Procesar variante
            cols = line.split("\t")
            if len(cols) < sample_idx + 1:
                # No hay columna para la muestra elegida
                print(f"No hay columna de muestra: {line}")
                continue

            chrom = cols[0]
            pos = cols[1]
            rsid = cols[2]
            ref = cols[3]
            alt = cols[4]
            filter = cols[6]
            info = cols[7]
            fmt = cols[8]
            fix_col = cols[sample_idx].strip()
            try:
              sample_field = cols[sample_idx +1].strip()
            except IndexError:
              sample_field = fix_col

            # Si no hay ID, podemos dejar '.' o construir algo como CHR:POS
            if rsid == "":
                # Aquí dejamos '.', pero podrías hacer:
                rsid = "."
            elif rsid.startswith("rs"):
                rsid = rsid
            else:
                rsid = "."

            # Cuando la longitud del alelo de referencia es mayor que 1, pasa a la siguiente linea
            if len(ref) > 1:
              continue

            # Comprueba la calidad. Si no es PASS pasa a la siguiente linea
            if filter.lower() != "pass":
                continue

            dosage = get_dosage_value(info)
            if (dosage > 0):
                a1_ref, a2_ref, a1, a2, dosage = ref, ".",alt,".",dosage
            else:
                # Buscar el índice de GT en FORMAT
                fmt_keys = fmt.split(":")
                if "GT" not in fmt_keys:
                    # Sin genotipo no podemos construir alelos
                    allele1, allele2 = ".", "."
                else:
                    print(line)
                    print(fix_col)
                    print(sample_field)
                    gt_index = fmt_keys.index("GT")
                    if (sample_field != "." and sample_field != ""):
                        sample_values = sample_field.split(":")
                        if gt_index >= len(sample_values):
                            a1_ref, a2_ref, a1, a2, dosage = ".", ".",".",".",0
                        else:
                            gt = sample_values[gt_index]
                            a1_ref, a2_ref, a1, a2, dosage = parse_genotype(gt, ref, alt, info)

            # cromosoma de salida
            if add_chr_prefix and not chrom.startswith("chr"):
                chr_out = f"chr{chrom}"
            else:
                chr_out = chrom

            genotipos_data.append({
                "rsID": rsid,
                "chr": chrom,
                "pos": pos,
                "allele1_ref": a1_ref,
                "allele2_ref": a2_ref,
                "allele1": a1,
                "allele2": a2,
                "dosage":dosage
            })

    # Crear el DataFrame de pandas con los datos acumulados
    df_genotipos = pd.DataFrame(genotipos_data)

    # Escribir el DataFrame estandarizado a un fichero TSV
    if (save_file):
        out_path = Path(out_vcf_path)
        df_genotipos.to_csv(out_vcf_path, sep="\t", index=False)

    return df_genotipos, out_vcf_path




In [ ]:
def leer_modelo_pgs(pgs_path, add_chr_prefix):
    """
    Lee el fichero de scoring del PGS Catalog y devuelve un diccionario:

        rsID -> {
            "chr": str,
            "pos": int,
            "effect_allele": str,
            "other_allele": str,
            "weight": float
        }
    """
    pgs = []
    header_cols = []
    col_indices = {} # To store mapping from desired_name -> index

    with extractData.open_compressed_file(pgs_path, "rt") as f:
        # Saltar los metadatos hasta la línea de cabecera
        for line in f:
            if line.startswith("rsID") or line.startswith("chr_name"): # Assuming this is the actual header line
                header_cols = [col.strip() for col in line.strip().split("\t")]
                print(f"Nombres de cabeceras --> {header_cols}")
                break

        # Check if header columns were found
        if not header_cols:
            print(f"Warning: Header line starting with 'rsID' not found in {pgs_path}", file=sys.stderr)
            return pd.DataFrame(pgs) # Return empty DataFrame if no header

        # Define mappings for desired column names to actual header names
        # and their original fixed indices as fallback
        desired_to_actual_cols = {
            "rsID": ["hm_rsID", "rsID"],
            "effect_allele": ["effect_allele", "effect_A", "allele_A"],
            "other_allele": ["other_allele", "other_A", "allele_B", "hm_inferOtherAllele"],
            "weight": ["weight", "beta", "score_w", "effect_weight"],
            "chr": ["hm_chr", "chromosome", "CHR", "chr_name"], # Original code implies index 9
            "pos": ["hm_pos", "position", "POS", "chr_position"] # Original code implies index 10
        }

        # Try to find the index for each desired column
        for desired_name, possible_actual_names in desired_to_actual_cols.items():
            found_idx = -1
            for actual_name in possible_actual_names:
                if actual_name in header_cols:
                    found_idx = header_cols.index(actual_name)
                    break

            # If not found by name, use the original positional index as fallback
            if found_idx == -1:
                if desired_name == "rsID": found_idx = 8
                elif desired_name == "effect_allele": found_idx = 3
                elif desired_name == "other_allele": found_idx = 4
                elif desired_name == "weight": found_idx = 5
                elif desired_name == "chr": found_idx = 9 # Based on original code
                elif desired_name == "pos": found_idx = 10 # Based on original code

                # If we still haven't found an index, it's a critical error for required columns
                if found_idx == -1:
                    print(f"Error: Could not determine index for column '{desired_name}' in file {pgs_path}", file=sys.stderr)
                    return pd.DataFrame(pgs) # Exit if essential column is missing

            col_indices[desired_name] = found_idx

        print(f"Col índices --> {col_indices}")

        # A partir de aquí vienen las filas de variantes
        for line in f:
            line = line.rstrip("\n")
            if not line:
                continue
            parts = line.split("\t")

            # Ensure the row has enough columns
            max_idx = -1
            if col_indices: # Ensure col_indices is not empty before calling max
                max_idx = max(col_indices.values())

            if len(parts) < max_idx + 1:
                continue

            # Access data using the determined indices
            rsid = parts[col_indices["rsID"]].strip()
            if rsid == "":
                rsid = "."

            chr_name = parts[col_indices["chr"]].strip()

            try:
                pos = int(parts[col_indices["pos"]])
            except (ValueError, IndexError):
                continue

            effect_allele = parts[col_indices["effect_allele"]].strip()
            other_allele = parts[col_indices["other_allele"]].strip()

            # Validate and convert weight
            try:
                weight = float(parts[col_indices["weight"]])
            except (ValueError, IndexError):
                continue


            # cromosoma de salida
            if add_chr_prefix and not chr_name.startswith("chr"):
                chr_out = f"chr{chr_name}"
            else:
                chr_out = chr_name

            pgs.append({
                "rsID": rsid,
                "chr": chr_out,
                "pos": pos,
                "effect_allele": effect_allele,
                "other_allele": other_allele,
                "weight": weight
            })

    # Crear el DataFrame de pandas con los datos acumulados
    df_pgs_modelo = pd.DataFrame(pgs)

    return df_pgs_modelo

In [ ]:
# -------------------------------------------------------------
# FUNCIÓN PARA CALCULAR EL PRS
# -------------------------------------------------------------
def calculate_prs(df, df_modelo):
    # If df_modelo is empty, return 0 PRS, as no scoring can be done
    if df_modelo.empty:
        return 0.0


    # Combina el modelo y los datos del participante por cromosoma y posición del cromosoma
    df_combinado_chr_pos = pd.merge(df, df_modelo, on=['chr', 'pos'], how='inner', suffixes=('_est', '_pgs'))

    # Calcula la columna 'score' condicionalmente
    df_combinado_chr_pos['score'] = np.where(
        (df_combinado_chr_pos['effect_allele'] == df_combinado_chr_pos['allele1']) |
        (df_combinado_chr_pos['effect_allele'] == df_combinado_chr_pos['allele2']),
        df_combinado_chr_pos['weight'] * df_combinado_chr_pos['dosage'],
        0 # Si la condición no se cumple, el score es 0
    )

    prs_total = df_combinado_chr_pos['score'].sum()

    return prs_total



# Paso 1. Estándarizar ficheros de datos

1.   Leer y cargar el fichero downloads_files.csv que contiene el listado de los ficheros a analizar y el tipo de datos.
2.   Recorrer el fichero y según el tipo de fichero ir a una función u otra para estandarizar (TSV, TSV3, VCF)
3. Para cada participante. Generar un fichero con sus datos estandarizados.




In [ ]:
# Procesando el fichero de variantes del individuo

# Abre el fichero Excel que contiene el listado de ficheros de partacipantes a estandarizar
files_to_std_df = extractData.load_csv_to_dataframe(data_path,download_files)

# Recorre uno a uno todos los elementos del dataset
print(f"##### ESTANDARIZACIÓN DE FICHEROS #####")

save_file = True
for index, row in files_to_std_df.iterrows():
    id_participant = row['Participant_ID']
    file_name = row['File_name']
    file_type = row['File_Type']
    var_file = participant_data_path + id_participant + "/" + file_name
    out_std_file = std_files +  id_participant + ".tsv"

    if index>=0:
        # Estandarizar fichero en función del tipo de archivo
        print(f"### {index}. PARTICIPANTE: {id_participant} {file_type} ###")
        if file_type == "TSV":
            variants_df, file_path = estandarizar_TSV_files(var_file, out_std_file, save_file)
        elif file_type == "VCF":
            variants_df, file_path = estandarizar_VCF_files(var_file, out_std_file, True, save_file)
        elif file_type == "TSV3":
            variants_df, file_path = estandarizar_TSV3_files(var_file, out_std_file, save_file)
        else:
            print(f"[{id_participant}] Tipo de fichero desconocido: {file_type}")





# Paso 2. Calcula el PRS para todos los participantes y todos los modelos

1. Recorre los ficheros estandarizdos de cada uno de los participatnes

2. Para cada participante

- Recorre todos los modelos

3. Para cada pareja participante - modelo calcula el PRS


Finalmente crea un dataframe donde las columnas son los modelos y las filas son los participantes.

In [ ]:
# 1. Crear una lista de nombres de ficheros en el directorio std_files
std_files_list = []
if os.path.exists(std_files):
    for filename in os.listdir(std_files):
        if os.path.isfile(os.path.join(std_files, filename)):
            std_files_list.append(filename)
print(f"Ficheros en {std_files}:\n{std_files_list[:5]}...") # Muestra los primeros 5 por brevedad
print(f"Total de ficheros estandarizados: {len(std_files_list)}\n")

# 2. Recorrer la carpeta Modelos y guardar nombres de carpeta y archivo
modelos_files_list = []
if os.path.exists(modelos):
    for root, dirs, files in os.walk(modelos):
        # Extraer el nombre de la carpeta actual (último componente de la ruta)
        current_folder_name = os.path.basename(root)
        for file in files:
            modelos_files_list.append((current_folder_name, file))
print(f"Modelos encontrados (carpeta, archivo):\n{modelos_files_list[:5]}...") # Muestra los primeros 5 por brevedad
print(f"Total de modelos encontrados: {len(modelos_files_list)}")

# 1. Extract participant IDs from std_files_list
participant_ids = [filename.replace('.tsv', '') for filename in std_files_list]

# 2. Extract unique model IDs from modelos_files_list
model_ids = sorted(list(set([model_info[0] for model_info in modelos_files_list])))

# 3. Create an empty Pandas DataFrame named results_df
results_df = pd.DataFrame(index=participant_ids, columns=model_ids)

print("results_df initialized with shape:", results_df.shape)
print("First 5 participant IDs (index):", results_df.index[:5].tolist())
print("First 5 model IDs (columns):", results_df.columns[:5].tolist())

# Iterate through each standardized participant file
for participant_filename in std_files_list:
    # Extract participant_id
    participant_id = participant_filename.replace('.tsv', '')

    # Construct full path to participant's standardized file
    current_participant_file = std_files + participant_filename

    # Load participant's data
    df_estandarizado = pd.read_csv(current_participant_file, sep='\t')

    # Iterate through each model file
    for model_id, model_filename in modelos_files_list:
        print(f"Processing participant {participant_id} with model {model_id}")

        # Construct full path to the model file
        current_pgs_path = modelos + model_id + "/" + model_filename

        # Load model data
        modelo_pgs = leer_modelo_pgs(current_pgs_path, True)

        # Calculate PRS
        prs_value = calculate_prs(df_estandarizado, modelo_pgs)

        # Store the calculated PRS value in results_df
        results_df.loc[participant_id, model_id] = prs_value

print("PRS calculation complete. Displaying the first few rows of results_df:")
print(results_df.head())

results_df.to_csv(data_path + "/prs_calculados.tsv", sep="\t", index=True)

# Análisis estadístico descriptivo

* Estandarización de los modelos
* Identificar correlaciones entre los modelos
* Estudiar la distribución de los diferentes modelos
* homocedasticidad
* Outlayers


In [ ]:
##### Estandarización de modelos a través de z-score #####

# Carga modelos
file_modelo_prs = data_path + "/prs_calculados.tsv"
file_model_prs_zscore = data_path + "/prs_calculados_zscore.tsv"
ID_COL = "Unnamed: 0"

df_prs = pd.read_csv(file_modelo_prs, sep="\t")

# Renombra la columna del modelo (Identificador)
df_prs = df_prs.rename(columns={df_prs.columns[0]: 'Participant_ID'})

# IDs de participantes a excluir
excluded_participants = ['huA06676', 'huDCF4FA']

# Filtrar el DataFrame para excluir a los participantes especificados
df_prs = df_prs[~df_prs['Participant_ID'].isin(excluded_participants)]

models = [c for c in df_prs.columns if c != 'Participant_ID']
X = df_prs[models].apply(pd.to_numeric, errors="coerce")

if X.isna().any().any():
    raise ValueError("Existen valores NaN en los PRS. Revisa el fichero.")

# Estandarización z-score
Xz = (X - X.mean(axis=0)) / X.std(axis=0, ddof=1)

# Reconstruye el dataset final
df_prs_z = pd.concat([df_prs[['Participant_ID']], Xz], axis=1)
df_prs_z.to_csv(file_model_prs_zscore, sep="\t", index=False)

In [ ]:
##### Estandarización de modelos a través de z-score #####

# Carga modelos
file_modelo_prs = data_path + "/prs_calculados.tsv"
file_model_prs_zscore = data_path + "/prs_calculados_zscore.tsv"
ID_COL = "Participant_ID"

df_prs = pd.read_csv(file_modelo_prs, sep="\t")

# Renombra la columna del modelo (Identificador)
df_prs = df_prs.rename(columns={df_prs.columns[0]: 'Participant_ID'})
models = [c for c in df_prs.columns if c != 'Participant_ID']
X = df_prs[models].apply(pd.to_numeric, errors="coerce")

if X.isna().any().any():
    raise ValueError("Existen valores NaN en los PRS. Revisa el fichero.")

# Estandarización z-score
Xz = (X - X.mean(axis=0)) / X.std(axis=0, ddof=1)

# Reconstruye el dataset final
df_prs_z = pd.concat([df_prs[['Participant_ID']], Xz], axis=1)
df_prs_z.to_csv(file_model_prs_zscore, sep="\t", index=False)

##### Correlación entre modelos #####
CORR_METHOD = "pearson"
CLUSTER_CORR_THRESHOLD = 0.90   # |r| >= 0.90 -> mismo clúster

# Figuras
HEATMAP_MAX_MODELS = 49        # si fueran muchos, podría recortarse
FIG_DPI = 160

ID_COL = 'Participant_ID'

models = [c for c in df_prs_z.columns if c != ID_COL]
Xz = df_prs_z[models].copy()

corr = Xz.corr(method=CORR_METHOD)
corr.to_csv(data_path + "/correlaciones_" + CORR_METHOD + ".csv")

# Histograma de correlaciones
# Excluir la diagonal
corr_matrix_without_diag = corr.copy()
np.fill_diagonal(corr_matrix_without_diag.values, np.nan)
corr_vals = corr_matrix_without_diag.stack().to_numpy()
plt.figure()
plt.hist(corr_vals, bins=40)
plt.title("Distribución de correlaciones Pearson entre modelos")
plt.xlabel("r")
plt.ylabel("frecuencia")
plt.tight_layout()
plt.show()

# Mapa de calor de correlaciones
plt.figure(figsize=(12, 10)) # Ajusta el tamaño de la figura según necesidad
plt.imshow(corr.values, cmap='coolwarm', aspect='auto')
plt.colorbar(label='Pearson r')
plt.xticks(range(len(models)), models, rotation=90, fontsize=8)
plt.yticks(range(len(models)), models, fontsize=8)
plt.title('Mapa de calor de correlaciones Pearson entre modelos')
plt.tight_layout()
plt.show()

# Distancia = 1 - |r|
dist = 1 - corr.abs()

# Convertir a forma condensada (requerido por scipy)
dist_condensed = squareform(dist.values, checks=False)

# 4) Clustering jerárquico
Z = linkage(dist_condensed, method="average")

# Umbral de distancia correspondiente a |r| >= threshold
distance_threshold = 1 - CLUSTER_CORR_THRESHOLD

clusters = fcluster(Z, t=distance_threshold, criterion="distance")

cluster_df = pd.DataFrame({
    "model": models,
    "cluster": clusters
}).sort_values("cluster")

cluster_df.to_csv(data_path + "/clusters_modelos_prs.csv", index=False)
print("OK: clusters_modelos_prs.csv")

# =========================
# 5) Modelo representativo por clúster
# =========================
representatives = []

for cl in sorted(cluster_df["cluster"].unique()):
    cl_models = cluster_df.loc[cluster_df["cluster"] == cl, "model"].tolist()

    if len(cl_models) == 1:
        representatives.append((cl, cl_models[0], 1, 1.0))
        continue

    # Correlación media absoluta dentro del clúster
    subcorr = corr.loc[cl_models, cl_models].abs()
    mean_corr = subcorr.mean(axis=1)

    best_model = mean_corr.idxmax()
    representatives.append(
        (cl, best_model, len(cl_models), mean_corr[best_model])
    )

rep_df = pd.DataFrame(
    representatives,
    columns=["cluster", "representative_model", "cluster_size", "mean_abs_correlation"]
).sort_values("cluster")

rep_df.to_csv(data_path + "/modelos_representativos_por_cluster.csv", index=False)
print("OK: modelos_representativos_por_cluster.csv")


# =========================
# 6) PRS estandarizados solo con modelos representativos
# =========================
rep_models = rep_df["representative_model"].tolist()

# Incluir el ID_COL (Participant_ID) junto con los modelos representativos
Xz_rep = df_prs_z[[ID_COL] + rep_models].copy()
Xz_rep.to_csv(data_path + "/prs_zscore_modelos_representativos.tsv", sep="\t", index=False)

print("OK: prs_zscore_modelos_representativos.tsv")
print(f"Reducción: {len(models)} → {len(rep_models)} modelos")


# 5.1 Heatmap de correlación (modelos originales o representativos)
# (Aquí lo hago con representativos para que sea legible)
corr_rep = Xz_rep[rep_models].corr(method="pearson") # Excluir ID_COL para la correlación

plt.figure(figsize=(10, 8))
plt.imshow(corr_rep.values, aspect="auto")
plt.colorbar(label="Pearson r")
plt.xticks(range(len(rep_models)), rep_models, rotation=90, fontsize=7)
plt.yticks(range(len(rep_models)), rep_models, fontsize=7)
plt.title("Heatmap de correlación (PRS z-score) - Modelos representativos")
plt.tight_layout()
plt.show()

# 5.2 Dendrograma (modelos originales, más informativo)
plt.figure(figsize=(12, 6))
dendrogram(Z, labels=models, leaf_rotation=90, color_threshold=distance_threshold)
plt.title(f"Dendrograma jerárquico de modelos PRS (dist = 1 - |r|), corte |r|≥{CLUSTER_CORR_THRESHOLD}")
plt.ylabel("Distancia (1 - |r|)")
plt.tight_layout()
plt.show() # Display the dendrogram

In [ ]:
# Identificar modelos excluidos
excluded_models = list(set(models) - set(rep_models))

print("##### Modelos Excluidos #####")
if excluded_models:
    print("Los siguientes modelos fueron excluidos por ser altamente correlacionados (Pearson |r| >= 0.90) con otros modelos y, por lo tanto, no se consideraron representativos de sus clústeres:")
    for model in sorted(excluded_models):
        print(f"- {model}")
    print(f"\nNúmero total de modelos excluidos: {len(excluded_models)}")
else:
    print("No se excluyó ningún modelo. Todos los modelos fueron considerados representativos.")

print("\n##### Explicación de la Exclusión #####")
print("Durante el proceso de clustering jerárquico, los modelos se agrupan en clústeres basados en su similitud (baja distancia, que corresponde a alta correlación Pearson). Para cada clúster, se selecciona un único 'modelo representativo' (medoid), que es el modelo con la mayor correlación media absoluta con los otros miembros de su clúster. Los demás modelos en ese clúster se consideran redundantes y se excluyen para reducir la dimensionalidad y evitar la multicolinealidad en análisis posteriores. Esto asegura que solo tengamos un conjunto de modelos relativamente independientes que representen la diversidad de los PRS calculados.")

In [ ]:
# Visualización de las distribuciones de los modelos (histogramas)
file_model_prs_zscore = data_path + "/prs_calculados_zscore.tsv"
df_prs_z = pd.read_csv(file_model_prs_zscore, sep="\t")

df_models =  pd.read_csv(data_path + "/modelos_representativos_por_cluster.csv", sep=",")
rep_df = df_models.sort_values("cluster")
rep_models = rep_df["representative_model"].tolist()


# Filtrar df_prs_z para incluir solo los modelos representativos
models_to_plot = [c for c in df_prs_z.columns if c in rep_models]

# Definir el número de filas y columnas para el grid
n_cols = 5
n_rows = (len(models_to_plot) + n_cols - 1) // n_cols # Calcula el número de filas necesario

fig, axes = plt.subplots(n_rows, n_cols, figsize=(n_cols * 4, n_rows * 3), sharex=False, sharey=False)
axes = axes.flatten() # Aplanar para facilitar la iteración

for i, model_name in enumerate(models_to_plot):
    ax = axes[i]

    # Plot histogram
    sns.histplot(df_prs_z[model_name], bins=30, kde=True, ax=ax, edgecolor='black', color='skyblue', stat='density')

    ax.set_title(f'{model_name}', fontsize=10)
    ax.set_xlabel('Valor Z-Score', fontsize=8)
    ax.set_ylabel('Densidad', fontsize=8)
    ax.tick_params(axis='x', labelsize=7)
    ax.tick_params(axis='y', labelsize=7)
    ax.grid(axis='y', alpha=0.75)

# Ocultar los subplots vacíos si los hay
for j in range(i + 1, len(axes)):
    fig.delaxes(axes[j])

plt.suptitle('Distribución Z-Score de los Modelos PRS con Línea de Densidad (Modelos Representativos)', fontsize=16)
plt.tight_layout(rect=[0, 0.03, 1, 0.97]) # Ajustar layout para suptitle
plt.savefig(f"{data_path}/hist_distribuciones_grid_representativos.png", dpi=300)
plt.show()
plt.close()

print("Histogramas de las distribuciones de los modelos representativos guardados en: " + data_path)

In [ ]:
ID_COL = 'Participant_ID'

# Usar solo los modelos representativos para el análisis
models = rep_models
Xz = df_prs_z[models].copy()

# Seguridad: asegurar numéricos y sin NaNs
Xz = Xz.apply(pd.to_numeric, errors="coerce")
if Xz.isna().any().any():
    raise ValueError("Hay NaNs tras convertir a numérico. Revisa separadores/valores.")

n, p = Xz.shape
print(f"Participantes: {n}, Modelos: {p}, Missing: {Xz.isna().sum().sum()}")

##### Descripción por modelo #####
desc_stats = Xz.describe().T
# Asimetria. Grado de la desvicación frente a la normal en sentido horizontal
desc_stats["skew"] = Xz.apply(skew)
# Desviación de las colas
desc_stats["kurtosis_excess"] = Xz.apply(lambda s: kurtosis(s, fisher=True))

#### Normalidad de los modelos ######
norm_rows = []
for col in models:
    vals = Xz[col].to_numpy()

    W, p_sh = shapiro(vals)             # Shapiro–Wilk (bien para n~123)

    norm_rows.append([col, W, p_sh])

norm_stats = pd.DataFrame(norm_rows, columns=["model", "shapiro_W", "shapiro_p"])
norm_stats = norm_stats.set_index('model')

# Corrección por múltiples contrastes (FDR)
rej, p_adj, *_ = multipletests(norm_stats["shapiro_p"], alpha=0.05, method="fdr_bh")
norm_stats["shapiro_p_fdr"] = p_adj
norm_stats["shapiro_reject_fdr"] = rej

# Combinar descriptivos y normalidad
combined_stats = desc_stats.merge(norm_stats, left_index=True, right_index=True, how='left')
combined_stats.to_csv(data_path + "/descriptivos_y_normalidad_por_modelo.csv")

# Modelos que no cumplen la distribución normal
non_normal_models = combined_stats[combined_stats["shapiro_reject_fdr"] == True]

if not non_normal_models.empty:
    print("Modelos que no cumplen la distribución normal (Shapiro-Wilk con FDR < 0.05):")
    display(non_normal_models[["shapiro_W", "shapiro_p", "shapiro_p_fdr"]])
else:
    print("Todos los modelos representativos cumplen con la distribución normal (Shapiro-Wilk con FDR < 0.05).")

# Boxplots de los modelos representativos en una única gráfica

# Filtrar df_prs_z para incluir solo los modelos representativos
# y luego preparar los datos para graficar todos los modelos en un mismo boxplot
models_data = df_prs_z[rep_models]

# Melt el DataFrame para tener un formato largo, adecuado para sns.boxplot con múltiples variables
models_melted = models_data.melt(var_name='Modelo PRS', value_name='Valor Z-Score')

plt.figure(figsize=(15, 7)) # Ajusta el tamaño de la figura para que sea legible
sns.boxplot(x='Modelo PRS', y='Valor Z-Score', data=models_melted, hue='Modelo PRS', legend=False, palette='viridis')
plt.title('Boxplots de todos los Modelos PRS (Z-Score) - Modelos Representativos', fontsize=16)
plt.xlabel('Modelo PRS', fontsize=10)
plt.ylabel('Valor Z-Score', fontsize=10)
plt.xticks(rotation=90, fontsize=8) # Rota las etiquetas del eje X para evitar solapamiento
plt.yticks(fontsize=8)
plt.grid(axis='y', alpha=0.75)
plt.tight_layout() # Ajusta el layout para asegurar que todo el contenido se muestre
plt.savefig(f"{data_path}/boxplots_todos_modelos_representativos.png", dpi=300)
plt.show()
plt.close()

print("Boxplots de todos los modelos representativos guardados en: " + data_path)

##### Homocedasticidad de los modelos #####
# Suposición de que la varianza de los errores es constante
groups = [Xz[c].to_numpy() for c in models]

lev_stat, lev_p = levene(*groups, center="median")
fl_stat, fl_p = fligner(*groups)
ba_stat, ba_p = bartlett(*groups)

homo = pd.DataFrame(
    [{
        "test": "Levene_median", "stat": lev_stat, "p": lev_p
    },{
        "test": "Fligner", "stat": fl_stat, "p": fl_p
    },{
        "test": "Bartlett", "stat": ba_stat, "p": ba_p
    }]
)
homo.to_csv(data_path + "/homocedasticidad_entre_modelos.csv", index=True)
print(homo)


##### Outliers / leverage / influyentes (participantes)  ######

# 5.1 Outliers multivariantes: Mahalanobis robusta
mcd = MinCovDet().fit(Xz)
md2 = mcd.mahalanobis(Xz)  # distancia^2
df_prs_z["md2_robust"] = md2

# 5.2 Leverage en espacio reducido (PCA al 95% varianza)
pca = PCA(n_components=0.95, svd_solver="full")
scores = pca.fit_transform(Xz.to_numpy())  # n x k
k = scores.shape[1]

XtX_inv = np.linalg.inv(scores.T @ scores)
h = np.einsum("ij,jk,ik->i", scores, XtX_inv, scores)  # diag(H)
df_prs_z["leverage_pca"] = h

# Umbral típico: 2k/n
lev_thr = 2 * k / n
df_prs_z["high_leverage"] = df_prs_z["leverage_pca"] > lev_thr

# 5.3 Score de influencia (heurístico) combinando distancia y leverage
df_prs_z["influence_score"] = df_prs_z["md2_robust"] * (df_prs_z["leverage_pca"] / (1 - df_prs_z["leverage_pca"] + 1e-9))

out_part = df_prs_z[[ID_COL, "md2_robust", "leverage_pca", "high_leverage", "influence_score"]].sort_values(
    "influence_score", ascending=False
)
out_part.to_csv(data_path + "/participantes_outliers_leverage_influencia.csv", index=False)
print("OK: participantes_outliers_leverage_influencia.csv")
display(out_part.head())

In [ ]:
file_auc_path = data_path + 'AUC.xlsx'
df_auc = pd.read_excel(file_auc_path)

import os
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

file_auc_path = data_path + 'AUC.xlsx'

# Check if the file exists before attempting to load
if os.path.exists(file_auc_path):
    df_auc = pd.read_excel(file_auc_path)

    print(f"'AUC.XLSX' loaded into df_auc with shape: {df_auc.shape}")
    print("First 5 rows of df_auc:")
    display(df_auc.head())
else:
    print(f"Error: The file '{file_auc_path}' was not found. Please ensure 'AUC.XLSX' is uploaded to the '{data_path}' directory.")
    df_auc = pd.DataFrame() # Initialize an empty DataFrame to avoid errors in subsequent steps

if df_auc.empty:
    print("Cannot calculate descriptive statistics: df_auc is empty. Please upload the 'AUC.XLSX' file first.")
else:
    # Assuming the AUC values are in a column named 'AUC'
    # If the column name is different, please adjust 'AUC' accordingly
    auc_column = 'AUC'

    if auc_column not in df_auc.columns:
        print(f"Error: Column '{auc_column}' not found in df_auc. Available columns are: {df_auc.columns.tolist()}")
    else:
        # Calculate descriptive statistics
        min_auc = df_auc[auc_column].min()
        max_auc = df_auc[auc_column].max()
        q1_auc = df_auc[auc_column].quantile(0.25)
        q3_auc = df_auc[auc_column].quantile(0.75)
        mean_auc = df_auc[auc_column].mean()
        median_auc = df_auc[auc_column].median()

        # Create a DataFrame for descriptive statistics
        descriptive_stats_df = pd.DataFrame({
            'Metric': ['Minimum', '1st Quartile', 'Median', 'Mean', '3rd Quartile', 'Maximum'],
            'Value': [min_auc, q1_auc, median_auc, mean_auc, q3_auc, max_auc]
        })

        print("\nDescriptive Statistics for AUC values:")
        display(descriptive_stats_df)

        # Generate horizontal boxplot
        plt.figure(figsize=(10, 4))
        sns.boxplot(x=df_auc[auc_column], orient='h', color='skyblue')
        plt.title('Boxplot de los valores de AUC')
        plt.xlabel('AUC Value')
        plt.grid(axis='x', linestyle='--', alpha=0.7)
        plt.tight_layout()
        plt.show()